Dưới đây là hướng dẫn song ngữ (🇬🇧 English + 🇻🇳 Vietnamese) cho **Chủ đề T2: Dự đoán hoạt tính sinh học từ các đặc trưng phân tử**, áp dụng **ChEMBL 35 + RDKit + PostgreSQL + AIMLOps Template**.

---

## 📘 T2\_1\_Extracting\_Bioactivity\_Data.ipynb

### 🇬🇧 Step 1: Extract 100 bioactivity records (IC50)

### 🇻🇳 Bước 1: Trích xuất 100 dòng dữ liệu hoạt tính sinh học (IC50)

### ✅ SQL code (run in pgAdmin)

```sql
-- Select bioactivity (IC50) with valid numeric values and SMILES
SELECT md.chembl_id,
       cs.canonical_smiles,
       act.standard_value::float AS ic50_nM,
       act.standard_type
FROM activities act
JOIN compound_structures cs ON act.molregno = cs.molregno
JOIN molecule_dictionary md ON md.molregno = cs.molregno
WHERE act.standard_type = 'IC50'
  AND act.standard_value ~ '^[0-9\.]+$'
  AND act.standard_value::float > 0
LIMIT 100;
```

> 🧠 Ghi chú:
>
> * Sử dụng `~` với `standard_value` yêu cầu ép kiểu đúng để tránh lỗi `operator does not exist: numeric ~ unknown`
> * Để an toàn, ép kiểu `standard_value::text ~ '^[0-9\.]+$'` hoặc dùng trực tiếp `standard_value::float`

---

## 📘 T2\_2\_QSAR\_Modeling.ipynb

### 🇬🇧 Step 2: Generate molecular descriptors and predict IC50

### 🇻🇳 Bước 2: Sinh đặc trưng phân tử và dự đoán IC50 bằng mô hình

### ✅ Python code

```python
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Define path
base_path = os.path.abspath(os.path.join(".."))
data_path = os.path.join(base_path, "data", "T2_ic50_sample.csv")  # File lưu từ SQL

# Load data
df = pd.read_csv(data_path)
df = df.dropna(subset=['canonical_smiles', 'ic50_nM'])

# Calculate descriptors
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)

# Prepare dataset
X = df[['MolWt', 'TPSA', 'LogP']]
y = df['ic50_nM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(f"R² score: {r2_score(y_test, y_pred):.3f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.3f}")
```

> 🧠 Lưu ý:
>
> * Nếu lỗi `squared=False` với scikit-learn cũ, thay bằng:
>
>   ```python
>   import numpy as np
>   rmse = np.sqrt(mean_squared_error(y_test, y_pred))
>   ```

---

## 📁 Cấu trúc file (AIMLOps Template)

```
project/
├── notebook/
│   ├── T2_1_Extracting_Bioactivity_Data.ipynb
│   └── T2_2_QSAR_Modeling.ipynb
├── data/
│   └── T2_ic50_sample.csv
├── src/
│   └── molecular_descriptors.py  # Optional
```

---

## 📌 Tổng kết | Summary

| Phần       | Nội dung chính                     | Mục đích     |
| ---------- | ---------------------------------- | ------------ |
| `T2_1_...` | Trích xuất IC50 và SMILES          | Tạo dataset  |
| `T2_2_...` | Sinh đặc trưng, huấn luyện mô hình | Dự đoán IC50 |

Bạn muốn tôi zip lại file mẫu và tạo file `.sql`, `.ipynb`, `.csv` đúng chuẩn cho bạn không?


In [11]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
# 1. Load data
base_path = os.path.abspath("..")
data_path = os.path.join(base_path, "data", "T2_ic50_sample.csv")
df = pd.read_csv(data_path)

# 2. Tạo đối tượng phân tử từ SMILES
df = df.dropna(subset=['canonical_smiles', 'ic50_nm'])
df['mol'] = df['canonical_smiles'].apply(Chem.MolFromSmiles)

# 3. Tính toán các descriptor hóa học
df['MolWt'] = df['mol'].apply(Descriptors.MolWt)
df['TPSA'] = df['mol'].apply(Descriptors.TPSA)
df['LogP'] = df['mol'].apply(Descriptors.MolLogP)

# 4. Huấn luyện mô hình QSAR
X = df[['MolWt', 'TPSA', 'LogP']]
y = df['ic50_nm']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. Đánh giá
# y_pred = model.predict(X_test)
# print(f"R² score: {r2_score(y_test, y_pred):.3f}")
# print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.3f}")

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"✅ R² score: {r2:.3f}")
print(f"✅ RMSE: {rmse:.3f}")


✅ R² score: 0.500
✅ RMSE: 20665.178
